# MC approaches

In [3]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Naive approach

In [27]:
def naive(
        n: int,            # number of Monte-Carlo simulations
        m: int,            # number of time steps
        r: float,          # risk-free interest rate
        sigma: float,      # volatility of the underlying
        K: float,          # strike of the fixed-strike Asian call
        T: float,          # maturity of the option
        S0: float          # spot at time t=0
) -> tuple[float, float]:  # returns the estimation and the half-length of the 95% confidence interval
    h = T/m
    dw = np.sqrt(h)*sigma*np.random.randn(m, n) # brownian motion increments x sigma (independent)
    wt = np.cumsum(dw, axis=0)                  # brownian motion values for time t = 1, ..., m
    w = np.insert(wt, obj=0, values=0, axis=0)  # add W0 = 0 at the top of each column -> shape (m + 1, n)
    drift = (r - sigma**2/2)*h*np.arange(m + 1)
    s = S0*np.exp(drift + w)                    # trajectory at time t = 0, ..., m
    m = s[:-1, ::].mean(axis=0)                 # estimatation of the mean value of the underlying
    c = np.exp(-r*T)*np.maximum(m - K, 0)       # price for each simulation
    return c.mean(), 1.96*c.std()/np.sqrt(n)

## 2. Improved Monte-Carlo

### 2.a Trapezoidal method

In [28]:
def trapeze(
        n: int,            # number of Monte-Carlo simulations
        m: int,            # number of time steps
        r: float,          # risk-free interest rate
        sigma: float,      # volatility of the underlying
        K: float,          # strike of the fixed-strike Asian call
        T: float,          # maturity of the option
        S0: float          # spot at time t=0
) -> tuple[float, float]:  # returns the estimation and the half-length of the 95% confidence interval
    h = T/m
    dw = np.sqrt(h)*sigma*np.random.randn(m, n) # brownian motion increments x sigma (independent)
    wt = np.cumsum(dw, axis=0)                  # brownian motion values for time t = 1, ..., m
    w = np.insert(wt, obj=0, values=0, axis=0)  # add W0 = 0 at the top of each column -> shape (m + 1, n)
    drift = (r - sigma**2/2)*h*np.arange(m + 1)
    s = S0*np.exp(drift + w)                    # trajectory at time t = 0, ..., m
    e = s[:-1, ::]*(1 + r*h/2 + wt/2)           # AMENDMENT HERE 
    m = e.mean(axis=0)                          # estimatation of the mean value of the underlying
    c = np.exp(-r*T)*np.maximum(m - K, 0)       # price for each simulation
    return c.mean(), 1.96*c.std()/np.sqrt(n)

### 2.b Conditional realization of the multiplicative correction factor

In [ ]:
def brownian_bridge_simulation(array: np.ndarray) -> np.ndarray:
    '''Take as input a (m + 1, n) array containing n trajectories of a Wiener process at m + 1 time steps
    and return a (m, n) array containing a realization of the integral of W_u - W_t_k between t_k and t_k+1
    for each k between 0 and m - 1 and each trajectory (see the fomrmulation of scheme 3)'''
    minc, n = array.shape
    m = minc - 1
    expectation = ... # formula to be completed
    stddev = ...      # formula to be completed
    return expectation + stddev*np.random.randn(m, n)

In [ ]:
def two_step_mc(
        n: int,            # number of Monte-Carlo simulations
        m: int,            # number of time steps
        r: float,          # risk-free interest rate
        sigma: float,      # volatility of the underlying
        K: float,          # strike of the fixed-strike Asian call
        T: float,          # maturity of the option
        S0: float          # spot at time t=0
) -> tuple[float, float]:  # returns the estimation and the half-length of the 95% confidence interval
    h = T/m
    dw = np.sqrt(h)*sigma*np.random.randn(m, n) # brownian motion increments x sigma (independent)
    wt = np.cumsum(dw, axis=0)                  # brownian motion values for time t = 1, ..., m
    w = np.insert(wt, obj=0, values=0, axis=0)  # add W0 = 0 at the top of each column -> shape (m + 1, n)
    drift = (r - sigma**2/2)*h*np.arange(m + 1)
    s = S0*np.exp(drift + w)                    # trajectory at time t = 0, ..., m
    real = brownian_bridge_simulation(w)
    # BELOW TO BE COMPLETED
    e = s[:-1, ::]*(1 + r*h/2 + wt/2)           # AMENDMENT HERE 
    m = e.mean(axis=0)                          # estimatation of the mean value of the underlying
    c = np.exp(-r*T)*np.maximum(m - K, 0)       # price for each simulation
    return c.mean(), 1.96*c.std()/np.sqrt(n)

## 3. Control variates

### 3.a With the naive method

### 3.b With the trapezoidal method

### 3.c With the "conditional realization" method

## Test (temporary)

In [20]:
a = np.arange(12).reshape((4, 3))
np.diff(a, prepend=0, axis=0)
a.cumsum(axis=0).mean(axis=0)

array([ 7.5, 10. , 12.5])

In [21]:
a = np.arange(12).reshape((4, 3))
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [25]:
np.insert(a, 0, 100, 0)[:-1, ::]

array([[100, 100, 100],
       [  0,   1,   2],
       [  3,   4,   5],
       [  6,   7,   8]])